# Problem Statement
What is the Circular Error Probable (CEP) for the upcoming drop test of the Electromechanical Recovery System (ERS) under a range of possible scenarios?

# Definition
Loosely speaking, CEP is the mean distance of a sample of points from their mean impact point. That is, a circle around the average point that contains 50% of the sample points. Here, we are defining CEP as the square root of the mean square error, i.e. the sum of range and azimuth variances and their covariance (ignoring bias for now).

# Situation
The ERS will be dropped out of a helicopter somewhere near the McMinnville Airport and for safety and liability reasons, we must have an expectation of the area it may land in. The primary variables we are considering is a range of possible helicopter trajectories (in terms of velocity and heading, with positions randomized around a certain mean) as well as a range of likely wind conditions (also as velocities and headings). GPS error is accounted for in the final calculation.

There are four operational modes (that is, three failure modes and a successful mission) to consider. There are also two helicopter airspeeds, 15.5 and 30 knots, to consider, thus we have eight cases of simulations in total.

For our operational modes, we consider the case that the mission is successful. We consider the two cases that only the drogue parachute fails and that the ERS system fails completely to be identical, given the distance we drop from. We consider the case that the drogue deploys correctly but remains attached while the main chute does not deploy. We consider the case that the main chute somehow deploys when the drogue is supposed to, which we expect to provide our worst case in terms of CEP spread.

In all scenarios, we consider the wind in a normal distribution about one direction with a standard deviation of 10 degrees. Thus, all simulations are contained within a 60 degree cone, and a large majority are within a 40 degree cone. The helicopter is considered to be going against that mean wind direction at the specified airspeed with some noise injected for realism. On the plots, the wind blows downwards and the helicopter flies upwards. It should also be noted that we may wish to discard any simulations with wind speeds above 5.15 m/s (that is approximately 10 knots) in order to lower CEP, but we will not do so here.

Since the simulations are conducted relative to an arbitrary direction, similar results should hold in any absolute direction by symmetry. The utility of this approach is that on the day of the test, given the current wind velocity, we should have an idea of how where the system is likely to drift to relative to its release point. This could allow us to select our release point to ensure that the possible impact zone is bounded, and in the case of an irregularly shaped field to drop in, we would have a better sense of which wind speeds are acceptable in any given direction.


# Methodology
Another Notebook contains a trajectory simulator and our documentation of it. We have reason to believe our model is approximately representative of the actual system and environment, assuming wind may be held constant for up to several minutes. The trajectory is available both as a numerical integration of a system of ordinary differential equations and as a linearization of that integration. In either case, the interaction of the drogue parachute with air must be integrated, but the phases prior to that and after that may be considered as steady-states without loss of significant precision. Hence, we use the numerical integration of our trajectory model in this Notebook which is much slower to compute, but much smoother.

Since CEP relies on observational evidence to calculate, we selected a Monte Carlo simulation of our trajectory model to generate data for statistical analysis. For each of our eight scenarios, we randomly sample 1,250 trajectories (with random variables considered independently) and we collect every final impact point, along with their initial conditions.

# Conclusions


In [1]:
%run drop_test_simulator.ipynb
import pandas as pd

from bokeh.io import output_file, output_notebook, show
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, LogColorMapper, BasicTicker, ColorBar,
    DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, Range1d
)
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5

In [2]:
def convert_to_lat(y):
    lat_in_ft = 364618.1186706298
    return y/lat_in_ft

def convert_to_long(x):
    long_in_ft = 257742.06079152823
    return x/long_in_ft

Note that this code has been written with a lot of generality to make it easier to specify later simulations of our various scenarios completely with single lines of code. 

In [3]:
heli_direction = 180
# note that if dt=0.25 the horizontal distance may be off by about 8 m (compared to dt=0.001)
def mc_init(num_sims=500, dt=0.25, mode='', airspeed=15.43334, wind_dir=0):
    all_coords = []
    v_wind_rndm = []
    v_plane_rndm = []
    init_pos_offset = []

    deg_wind_mean, deg_wind_std_dev  = (wind_dir, 10) # 20-40-60 degree cone (with probability 68-95-99.7)
    
    for i in range(num_sims):
        v_wind_rndm.append([np.random.rayleigh(2.47344), # rayleigh distro good for wind magnitude
                            np.random.normal(deg_wind_mean, deg_wind_std_dev)])
        
        v_plane_rndm.append(np.random.normal(
                airspeed + v_wind_rndm[-1][0] * np.cos(np.radians(v_wind_rndm[-1][1] - heli_direction)),
                1))
        
        init_pos_offset.append([np.random.normal(0, 30 * 0.3048), # converting feet to meters
                                np.random.normal(0, 30 * 0.3048),
                                np.random.normal(0, 15 * 0.3048)]) # account for the imperfections of reality
    print('random wind and helicopter vectors obtained\n')

    for i, vw in enumerate(v_wind_rndm):
        all_coords.append(trajectory(v_plane=v_plane_rndm[i],
                                          deg_plane=heli_direction,
                                          v_wind=vw[0],
                                          deg_wind=vw[1],
                                          x_0_offset=init_pos_offset[i],
                                          mode=mode,
                                          dt=dt)[0])
        if i % 250 == 0: print('iterations:', i)
    print('done simulations!\n')
    
    return (all_coords, v_wind_rndm, v_plane_rndm)

In [4]:
def polar_coord(df):
    # magnitude of landing coordinate
    df['norm'] = df.loc[:,['x','y']].apply(np.linalg.norm, axis=1, raw=True)
    # azimuth angle of landing (note that this is on the unit circle)
    df['theta'] = df.apply(lambda pos: np.arctan2(pos[1], pos[0]), axis=1, raw=True)

def lat_long(df):
    df['latitude'] = df.apply(lambda tmp: convert_to_lat(tmp[1]), axis=1, raw=True)
    df['longitude'] = df.apply(lambda tmp: convert_to_long(tmp[0]), axis=1, raw=True)
      
def handle_data(all_coords, v_wind_rndm, v_plane_rndm, wind_limit=None):
    def wind_envelope(df, cutoff):
        bool_list = df['v_w'] <= cutoff
        return df[bool_list]
    # landing coordinates
    results_df = pd.DataFrame.from_records([rotate(np.array([m_to_ft(Xi) for Xi in X]),
                                                   np.radians(heli_direction) - np.pi/2)
                                            for X in all_coords],
                                           columns=['x', 'y'])
    #results_df = pd.DataFrame.from_records([np.array([m_to_ft(Xi) for Xi in X])
    #                                        for X in all_coords],
    #                                       columns=['x','y'])
    polar_coord(results_df)
    # plane velocities
    results_df['v_plane'] = [ms_to_knots(vel) for vel in v_plane_rndm]
    # wind magnitude
    results_df['v_w'] = [ms_to_knots(vel[0]) for vel in v_wind_rndm]
    # wind direction (is this on unit circle? need to think more...)
    results_df['deg_w'] = [vel[1] for vel in v_wind_rndm]
    lat_long(results_df)
    
    if wind_limit != None:
        print("Cutting out all runs with winds above "+str(wind_limit)+" m/s.")
        results_df = wind_envelope(results_df, wind_limit)
        
    print(results_df.describe())
    return results_df

In [5]:
amount = 1000
# mission success
nominal_against_coords, nominal_against_wind, nominal_against_plane = mc_init(num_sims=amount, dt=0.05, airspeed=15.43334, wind_dir=0)

# main chute failure
nomain_against_coords, nomain_against_wind, nomain_against_plane = mc_init(num_sims=amount, dt=0.05, mode='A', airspeed=15.43334, wind_dir=0)

# drogue failure
nodrogue_against_coords, nodrogue_against_wind, nodrogue_against_plane = mc_init(num_sims=amount, dt=0.05, mode='B', airspeed=15.43334, wind_dir=0)

# early deployment
early_against_coords, early_against_wind, early_against_plane = mc_init(num_sims=amount, dt=0.05, mode='C', airspeed=15.43334, wind_dir=0)

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!



In [6]:
# mission success
nominal_across_coords, nominal_across_wind, nominal_across_plane = mc_init(num_sims=amount, dt=0.05, airspeed=15.43334, wind_dir=90)

# main chute failure
nomain_across_coords, nomain_across_wind, nomain_across_plane = mc_init(num_sims=amount, dt=0.05, mode='A', airspeed=15.43334, wind_dir=90)

# drogue failure
nodrogue_across_coords, nodrogue_across_wind, nodrogue_across_plane = mc_init(num_sims=amount, dt=0.05, mode='B', airspeed=15.43334, wind_dir=90)

# early deployment
early_across_coords, early_across_wind, early_across_plane = mc_init(num_sims=amount, dt=0.05, mode='C', airspeed=15.43334, wind_dir=90)

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!

random wind and helicopter vectors obtained

iterations: 0
iterations: 250
iterations: 500
iterations: 750
done simulations!



# Data Handling for Against Wind Simulations

In [7]:
nominal_against_data = handle_data(nominal_against_coords, nominal_against_wind, nominal_against_plane, wind_limit=None)
nomain_against_data = handle_data(nomain_against_coords, nomain_against_wind, nomain_against_plane, wind_limit=None)
nodrogue_against_data = handle_data(nodrogue_against_coords, nodrogue_against_wind, nodrogue_against_plane, wind_limit=None)
early_against_data = handle_data(early_against_coords, early_against_wind, early_against_plane, wind_limit=None)

                 x            y         norm        theta      v_plane  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean    590.398555     3.570249   614.324668     0.022504    24.062272   
std     423.743980   152.063739   416.943526     0.669579     3.862584   
min    -183.373952  -743.692655     6.839429    -3.120781    11.106102   
25%     269.159002   -77.026518   286.228000    -0.177936    21.570308   
50%     529.281097    -1.321278   542.258944    -0.003072    24.412641   
75%     853.794373    86.379627   882.064023     0.190277    26.816116   
max    2174.333985   685.629721  2177.058199     3.111734    33.753245   

               v_w        deg_w     latitude    longitude  
count  1000.000000  1000.000000  1000.000000  1000.000000  
mean      6.012178     0.248264     0.000010     0.002291  
std       3.216872    10.190387     0.000417     0.001644  
min       0.085280   -33.272231    -0.002040    -0.000711  
25%       3.566501    -6.908696  

# Data Handling for Across Wind Simulations

In [8]:
nominal_across_data = handle_data(nominal_across_coords, nominal_across_wind, nominal_across_plane, wind_limit=None)
nomain_across_data = handle_data(nomain_across_coords, nomain_across_wind, nomain_across_plane, wind_limit=None)
nodrogue_across_data = handle_data(nodrogue_across_coords, nodrogue_across_wind, nodrogue_across_plane, wind_limit=None)
early_across_data = handle_data(early_across_coords, early_across_wind, early_across_plane, wind_limit=None)

                 x            y         norm        theta      v_plane  \
count  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000   
mean   -213.835210   736.338148   787.933940     1.926675    30.126152   
std     159.283387   390.099220   380.256647     0.275888     2.189562   
min    -944.079489    12.741736   159.143935     1.217643    21.744051   
25%    -289.947982   443.287237   502.893132     1.750197    28.587981   
50%    -209.248394   695.505848   744.939519     1.900211    30.110107   
75%    -133.757006   962.351558  1002.015424     2.072311    31.631679   
max     492.516510  2377.034331  2380.430339     3.084899    37.968512   

               v_w        deg_w     latitude    longitude  
count  1000.000000  1000.000000  1000.000000  1000.000000  
mean      5.786620    90.755515     0.002019    -0.000830  
std       3.048772    10.179162     0.001070     0.000618  
min       0.139352    56.773432     0.000035    -0.003663  
25%       3.477388    83.973440  

## Saving data and Aggregate Results

In [9]:
nominal_against_data.to_csv(path_or_buf='./drop_test_sample_data/nominal_against_data.csv')
nomain_against_data.to_csv(path_or_buf='./drop_test_sample_data/nomain_against_data.csv')
nodrogue_against_data.to_csv(path_or_buf='./drop_test_sample_data/nodrogue_against_data.csv')
early_against_data.to_csv(path_or_buf='./drop_test_sample_data/early_against_data.csv')

In [10]:
nominal_across_data.to_csv(path_or_buf='./drop_test_sample_data/nominal_across_data.csv')
nomain_across_data.to_csv(path_or_buf='./drop_test_sample_data/nomain_across_data.csv')
nodrogue_across_data.to_csv(path_or_buf='./drop_test_sample_data/nodrogue_across_data.csv')
early_across_data.to_csv(path_or_buf='./drop_test_sample_data/early_across_data.csv')